# Computing NGS coverage

![ngs sequencing](http://www.genohm.com/wp-content/uploads/2015/05/DNA-actg1.jpg)

## Understanding the data format and specifications

Take a look with the notebook editor to the subset file we have!

```python
HEADER_CHAR = "@"
SEP = ":"

# Illumina 1.8+ Phred+33,  raw reads typically (0, 41)
PHRED_INIT = 33
# minimum quality
PHRED_THOLD = 20  
# strand
FORWARD_STRAND_FLAG = 16
```

Writing constants to a separate module

In [5]:
%%writefile $ngs_module

""" NGS functions """

# Symbol for header inside SAM format
HEADER_CHAR = "@"
# Chromosomic separator
SEP = ":"
# Formats for quality reading: http://j.mp/1zo30bt
# Illumina 1.8+ Phred+33,  raw reads typically (0, 41)
PHRED_INIT = 33
# What is the minimum quality?
PHRED_THOLD = 20  
# Handle STRAND:
FORWARD_STRAND_FLAG = 16


Overwriting ./tmp/ngs.py


Handling the strand

In [6]:
%%writefile -a $ngs_module


def compute_strand(flag, myseq='ACTG', mystart=1):
    """ Strand is the bit of information that specify the direction of DNA reading """
    
    # Convert the flag to decimal, and check
    # the bit in the 5th position from the right.
    if flag & FORWARD_STRAND_FLAG:       # Strand forward
        mystop = mystart + len(myseq)
    else:               # Strand reverse
        mystop = mystart + 1
        mystart = mystop - len(myseq)

    return mystart, mystop


Appending to ./tmp/ngs.py


Ordering chromosomes...

In [27]:
test_list = ["chr21", "chr2", "chrx", "chr10", "chrm", "chr1"]
test_list.sort()
test_list

['chr1', 'chr10', 'chr2', 'chr21', 'chrm', 'chrx']

In [28]:
# If the chromosome is a number (digit)
shortchr = test_list[2][3:]
shortchr.zfill(2)

'02'

In [29]:
# If the chromosome is an alphabet letter
shortchr = test_list[4][3:]
code = ord(shortchr).__str__()
code

'109'

In [7]:
%%writefile -a $ngs_module

def ngs_split(line=None):
    """ Recover necessary data for NGS analysis """
    
    pieces = line.split("\t")

    myflag = int(pieces[1])
    mychr = pieces[2]
    mystart = int(pieces[3])
    myseq = pieces[9]
    myqc = pieces[10]
    mystart, mystop = compute_strand(myflag, myseq, mystart)

    shortchr = mychr[3:]
    if shortchr.isdigit():
        code = shortchr.zfill(2)  # sortable chromosome code
    else:
        code = ord(shortchr).__str__()

    return mychr, mystart, mystop, myseq, myflag, myqc, code


Appending to ./tmp/ngs.py


Make the module loadable and test it with import

In [8]:
! touch ./tmp/__init__.py
from tmp.ngs import ngs_split

In [9]:
ngs_split(mline)

('chrM',
 728,
 803,
 'CACCCCCACGGGAAACAGCAGTGATTAACCTTTAGCAATAAACGAAAGTTTAACTAAGCTATACTAACCCTGTCT',
 161,
 'CCCFFFFFHHHGHJJIJGIJJJIGIJJJJIIJJJJJJEIJJIJGI=FHGIHHGGIIGIHBHFGHFFFFFDECCCC',
 '77')

In [10]:
ngs_split(oline)

('chr1',
 142638010,
 142638085,
 'AGACAGGCTTGTCCTCTGGCCACAAACTATTAACGTTTCTCCCACATGGAAATTATGCTTTGCCCCTCTCAAGAG',
 121,
 '>;@DCBB@;7=;HEIHE@F@C@IGGGIGFBGJIIHDG?B?AGHDGFEIGGJJJIIGIIIHGIHDFHFFDFFFC@@',
 '01')

We can check with nb editor the final result

In [11]:
from tmp.ngs import SEP, PHRED_INIT, PHRED_THOLD

# recover data needed from one line
mychr, mystart, mystop, myseq, _, myqc, _ = ngs_split(oline)

# Cycle
for i in range(mystart, mystop):
    mypos = i - mystart
    current_qc = ord(myqc[mypos]) - PHRED_INIT  # quality value
    if current_qc > PHRED_THOLD:
        label = mychr + SEP + i.__str__()
        #label = code + SEP + i.__str__() + SEP + mychr
        current_letter = myseq[mypos]
        print(label, current_letter)

chr1:142638010 A
chr1:142638011 G
chr1:142638012 A
chr1:142638013 C
chr1:142638014 A
chr1:142638015 G
chr1:142638016 G
chr1:142638017 C
chr1:142638018 T
chr1:142638019 T
chr1:142638020 G
chr1:142638021 T
chr1:142638022 C
chr1:142638023 C
chr1:142638024 T
chr1:142638025 C
chr1:142638026 T
chr1:142638027 G
chr1:142638028 G
chr1:142638029 C
chr1:142638030 C
chr1:142638031 A
chr1:142638032 C
chr1:142638033 A
chr1:142638034 A
chr1:142638035 A
chr1:142638036 C
chr1:142638037 T
chr1:142638038 A
chr1:142638039 T
chr1:142638040 T
chr1:142638041 A
chr1:142638042 A
chr1:142638043 C
chr1:142638044 G
chr1:142638045 T
chr1:142638046 T
chr1:142638047 T
chr1:142638048 C
chr1:142638049 T
chr1:142638050 C
chr1:142638051 C
chr1:142638052 C
chr1:142638053 A
chr1:142638054 C
chr1:142638055 A
chr1:142638056 T
chr1:142638057 G
chr1:142638058 G
chr1:142638059 A
chr1:142638060 A
chr1:142638061 A
chr1:142638062 T
chr1:142638063 T
chr1:142638064 A
chr1:142638065 T
chr1:142638066 G
chr1:142638067 C
chr1:142638068

In [12]:
%%writefile $myfile

from mrjob.job import MRJob
try:
    from ngs import ngs_split, PHRED_INIT, PHRED_THOLD, HEADER_CHAR, SEP
except:
    pass

class MRcoverage(MRJob):
    """ Map Reduce for NGS coverage computation"""

    def mapper(self, _, line):
        """ Split data into keys and value """

        if line[0] == HEADER_CHAR:
            yield "header", 1
        else:
            # Get data
            mychr, mystart, mystop, myseq, myflag, myqc, code = ngs_split(line)
            # For each base of my dna sequence
            for i in range(mystart, mystop):
                mypos = i - mystart
                current_qc = ord(myqc[mypos]) - PHRED_INIT  # quality value
                if current_qc > PHRED_THOLD:
                    # Choose the key to emit
                    label = code + SEP + i.__str__() + SEP + mychr
                    current_letter = myseq[mypos]
                    yield label, 1
                    yield label + SEP + current_letter, 1

    def reducer(self, key, values):
        """ Sum up values"""
        yield key, sum(values)

if __name__ == '__main__':
    MRcoverage().run()


Overwriting ./tmp/job.py


In [13]:
# Launch the job (not on Hadoop)

output_file = mydir + "out.txt"

! python $myfile $myshortinput 1> $output_file


Using configs in /etc/mrjob.conf
Creating temp directory /tmp/job.jovyan.20160316.095320.214594
Running step 1 of 1...
Streaming final output from /tmp/job.jovyan.20160316.095320.214594/output...
Removing temp directory /tmp/job.jovyan.20160316.095320.214594...


In [14]:
import os

def get_output(outfile):
    """ Get Hadoop results from MrJob output file """
    
    data = {}

    if os.path.exists(outfile):
        with open(outfile) as out:
            outstring = out.read()
        for line in outstring.split('\n'):
            if line.strip() == '':
                continue
            tmp = line.split('\t')
            data[tmp[0].strip('"')] = int(tmp[1])

    return data


In [15]:
get_output(output_file)

{'01:142594950:chr1': 1,
 '77:5041:chrM:A': 3,
 '01:53324360:chr1': 1,
 '77:7597:chrM': 4,
 '77:7389:chrM:A': 2,
 '77:14928:chrM:A': 2,
 '01:25742934:chr1:C': 1,
 '77:4870:chrM:A': 6,
 '01:142712076:chr1': 2,
 '01:228338:chr1:G': 6,
 '77:7418:chrM': 1,
 '01:804248:chr1:C': 2,
 '01:1844948:chr1:T': 1,
 '77:4183:chrM': 6,
 '01:142641071:chr1': 1,
 '01:142692108:chr1': 3,
 '01:142540948:chr1': 4,
 '01:142540987:chr1': 5,
 '77:8794:chrM': 3,
 '01:142543692:chr1:T': 1,
 '77:15495:chrM:C': 4,
 '01:142691529:chr1': 1,
 '77:12145:chrM:A': 3,
 '01:112838955:chr1:C': 1,
 '77:11608:chrM': 3,
 '01:52552737:chr1': 2,
 '77:14825:chrM:T': 3,
 '77:13712:chrM': 5,
 '01:142554366:chr1:G': 1,
 '77:1525:chrM': 5,
 '01:76324720:chr1:T': 1,
 '01:142547369:chr1': 1,
 '01:142560636:chr1:A': 1,
 '01:142611977:chr1': 1,
 '77:9525:chrM': 1,
 '01:142657707:chr1': 1,
 '77:11371:chrM:T': 2,
 '77:4312:chrM:A': 1,
 '77:8880:chrM:G': 2,
 '01:142553681:chr1:G': 1,
 '77:1473:chrM': 4,
 '77:5075:chrM': 3,
 '01:142548325:

In [16]:
def sortdict(mydict):
    """ Sort in descend based on values count """
    
    return sorted(mydict.items(), key=lambda x: x[1], reverse=True) 


In [17]:
sortdict(get_output(output_file))

[('header', 97),
 ('01:228373:chr1:T', 28),
 ('01:228377:chr1', 28),
 ('01:228376:chr1', 28),
 ('01:228381:chr1:C', 28),
 ('01:228374:chr1', 28),
 ('01:228373:chr1', 28),
 ('01:228372:chr1', 28),
 ('01:228371:chr1', 28),
 ('01:228371:chr1:T', 28),
 ('01:228375:chr1', 28),
 ('01:228380:chr1:T', 28),
 ('01:228379:chr1', 28),
 ('01:228380:chr1', 28),
 ('01:228381:chr1', 28),
 ('01:228376:chr1:C', 28),
 ('01:228374:chr1:G', 28),
 ('01:228379:chr1:C', 28),
 ('01:228378:chr1:T', 28),
 ('01:228378:chr1', 28),
 ('01:228372:chr1:A', 28),
 ('01:228375:chr1:C', 28),
 ('01:228377:chr1:T', 28),
 ('01:228362:chr1', 27),
 ('01:228382:chr1:T', 27),
 ('01:228361:chr1', 27),
 ('01:228370:chr1:A', 27),
 ('01:228370:chr1', 27),
 ('01:228359:chr1', 27),
 ('01:228363:chr1', 27),
 ('01:228382:chr1', 27),
 ('01:228360:chr1', 27),
 ('01:228358:chr1', 27),
 ('01:228368:chr1', 26),
 ('01:228364:chr1:T', 26),
 ('01:228365:chr1', 26),
 ('01:228369:chr1:C', 26),
 ('01:228366:chr1', 26),
 ('01:228364:chr1', 26),
 ('

## One problem to deal with

# NOT TRUE!

use the --file option !!

Using a separate module in MrJob does not work (like with normal Python)

In [18]:
! python $myfile -r hadoop $myshortinput 1> log.out 2> log.err

In [19]:
! cat log.err

Using configs in /etc/mrjob.conf
Using Hadoop version 2.6.0
Copying local files to hdfs:///user/jovyan/tmp/mrjob/job.jovyan.20160316.095704.267325/files/...
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar4111945800521108955/] [] /tmp/streamjob1427610212578107957.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1458118988031_0001
  Submitted application application_1458118988031_0001
  The url to track the job: http://95a1ff171805:8088/proxy/application_1458118988031_0001/
  Running job: job_1458118988031_0001
  Job job_1458118988031_0001 running in uber mode : false
   map 0% reduce 0%
  Task Id : attempt_1458118988031_0001_m_000001_0, Status : FAILED
Error: java.lang.RuntimeException: PipeMapRed.waitOutputThreads(): subprocess failed with code 1
	at org.apache.hadoop.streaming.PipeMapRed.waitOutputThreads(PipeMapRed.java:3

If you think of how Hadoop Streaming works, you have to send every file to Hadoop :)

Hint: your modules could be compressed and sent to HADOOP or EMR

http://mrjob.readthedocs.org/en/latest/guides/setup-cookbook.html#putting-your-source-tree-in-pythonpath

In [21]:
# A possible solution is to combine module and file together
! cat $ngs_module $myfile > tmp/hadoop.py

In [22]:
! python tmp/hadoop.py -r hadoop $myshortinput 1> log.out 2> log.err

In [23]:
get_output("log.out")

{'01:142594950:chr1': 1,
 '77:5041:chrM:A': 3,
 '01:53324360:chr1': 1,
 '77:7597:chrM': 4,
 '77:7389:chrM:A': 2,
 '77:14928:chrM:A': 2,
 '01:25742934:chr1:C': 1,
 '77:4870:chrM:A': 6,
 '01:142712076:chr1': 2,
 '01:228338:chr1:G': 6,
 '77:7418:chrM': 1,
 '01:804248:chr1:C': 2,
 '01:1844948:chr1:T': 1,
 '77:4183:chrM': 6,
 '01:142641071:chr1': 1,
 '01:142692108:chr1': 3,
 '01:142540948:chr1': 4,
 '01:142540987:chr1': 5,
 '77:8794:chrM': 3,
 '01:142543692:chr1:T': 1,
 '77:15495:chrM:C': 4,
 '01:142691529:chr1': 1,
 '77:12145:chrM:A': 3,
 '01:112838955:chr1:C': 1,
 '77:11608:chrM': 3,
 '01:52552737:chr1': 2,
 '77:14825:chrM:T': 3,
 '77:13712:chrM': 5,
 '01:142554366:chr1:G': 1,
 '77:1525:chrM': 5,
 '01:76324720:chr1:T': 1,
 '01:142547369:chr1': 1,
 '01:142560636:chr1:A': 1,
 '01:142611977:chr1': 1,
 '77:9525:chrM': 1,
 '01:142657707:chr1': 1,
 '77:11371:chrM:T': 2,
 '77:4312:chrM:A': 1,
 '77:8880:chrM:G': 2,
 '01:142553681:chr1:G': 1,
 '77:1473:chrM': 4,
 '77:5075:chrM': 3,
 '01:142548325:

## Counters

In [24]:
%%writefile $myfile

from mrjob.job import MRJob
from ngs import ngs_split, PHRED_INIT, PHRED_THOLD, HEADER_CHAR, SEP

class MRcoverage(MRJob):

    def mapper(self, _, line):
        if line[0] != HEADER_CHAR:
            mychr, mystart, mystop, myseq, myflag, myqc, code = ngs_split(line)
            # Save chromosome info
            self. inc('chromosomes', mychr, 1)

            for i in range(mystart, mystop):
                mypos = i - mystart
                current_qc = ord(myqc[mypos]) - PHRED_INIT  # quality value
                if current_qc > PHRED_THOLD:
                    yield mychr + SEP + str(i) + SEP + myseq[mypos], 1

    def reducer(self, key, values):
        """ Sum up values"""
        yield key, sum(values)

if __name__ == '__main__':
    MRcoverage().run()


Overwriting ./tmp/job.py


In [25]:
! python $myfile $myshortinput 1> log.out 2> log.err

In [26]:
cat log.err

Using configs in /etc/mrjob.conf
Creating temp directory /tmp/job.jovyan.20160316.100554.118699
Running step 1 of 1...
Counters: 2
	chromosomes
		chr1=500
		chrM=903
Counters: 2
	chromosomes
		chr1=500
		chrM=903
Streaming final output from /tmp/job.jovyan.20160316.100554.118699/output...
Removing temp directory /tmp/job.jovyan.20160316.100554.118699...


# End of Chapter